In [1]:
import importlib
import sys
import torch
import pickle
import os
from tqdm.notebook import tqdm

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM


In [2]:
# Load model
file_path_model = '../../../training_variational_dropout/Helpdesk/Helpdesk_full_no_grad_norm_new_2.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../encoded_data/helpdesk_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

print(f"Model loaded")
print(f"Dataset loaded: {len(bpic_17_test_dataset)} cases")


Data set categories:  ([('Activity', 16, {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'EOS': 5, 'INVALID': 6, 'Insert ticket': 7, 'RESOLVED': 8, 'Require upgrade': 9, 'Resolve SW anomaly': 10, 'Resolve ticket': 11, 'Schedule intervention': 12, 'Take in charge ticket': 13, 'VERIFIED': 14, 'Wait': 15}), ('Resource', 24, {'EOS': 1, 'Value 1': 2, 'Value 10': 3, 'Value 11': 4, 'Value 12': 5, 'Value 13': 6, 'Value 14': 7, 'Value 15': 8, 'Value 16': 9, 'Value 17': 10, 'Value 18': 11, 'Value 19': 12, 'Value 2': 13, 'Value 20': 14, 'Value 21': 15, 'Value 22': 16, 'Value 3': 17, 'Value 4': 18, 'Value 5': 19, 'Value 6': 20, 'Value 7': 21, 'Value 8': 22, 'Value 9': 23}), ('Variant index', 166, {'1.0': 1, '10.0': 2, '100.0': 3, '101.0': 4, '102.0': 5, '103.0': 6, '104.0': 7, '105.0': 8, '106.0': 9, '107.0': 10, '108.0': 11, '109.0': 12, '11.0': 13, '110.0': 14, '111.0': 15, '112.0': 16, '113.0': 17, '114.0': 18, '12.0': 19, '13.0': 20, '14.0': 21, '15.0': 22, '16.0

In [3]:
attack_dataset = '../../../../../encoded_data/NEW_last_event_attack_clean.pkl'
predefined_dataset = torch.load(attack_dataset, weights_only=False)

 

In [4]:
# Import and reload the adversarial attack module
import evaluation.adversarial_attack
importlib.reload(evaluation.adversarial_attack)
from evaluation.adversarial_attack import GradientAscentAttacker

# Create the gradient ascent attacker
attacker = GradientAscentAttacker(
    model=model,
    dataset=bpic_17_test_dataset,
    concept_name='Activity',
    growing_num_values=['case_elapsed_time'],
    dataset_predefined_prefixes=predefined_dataset
)

print("GradientAscentAttacker initialized")


GradientAscentAttacker initialized


In [5]:
# Configure attack parameters
max_iterations = 1  # Maximum gradient ascent steps per attack
step_size = 0.05      # Learning rate for gradient ascent
epsilon = 10000.0         # Maximum allowed perturbation (L_inf norm) previous 0.15
early_stop = True     # Stop when prediction becomes wrong

print(f"Attack parameters:")
print(f"  Max iterations: {max_iterations}")
print(f"  Step size: {step_size}")
print(f"  Epsilon: {epsilon}")
print(f"  Early stop: {early_stop}")


Attack parameters:
  Max iterations: 1
  Step size: 0.05
  Epsilon: 10000.0
  Early stop: True


In [6]:
# Function to save results in chunks
def save_chunk(results, chunk_number):
    filename = os.path.join(output_dir, f'gradient_ascent_attack_part_{chunk_number:04d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

# Set output directory
output_dir = '../../../../../evaluation_results/robustness/Helpdesk/gradient_ascent_attack/'
os.makedirs(output_dir, exist_ok=True)

save_every = 50  # Save every N successful attacks
print(f"Output directory: {output_dir}")
print(f"Saving every {save_every} attacks")


Output directory: ../../../../../evaluation_results/robustness/Helpdesk/gradient_ascent_attack/
Saving every 50 attacks


In [7]:
# Perform gradient ascent attacks on all predefined prefixes
print("Starting gradient ascent attacks...")
print(f"Total prefix-suffix pairs to attack: {len(predefined_dataset)}")

results = attacker.attack_predefined_prefixes(
    max_iterations=max_iterations,
    step_size=step_size,
    epsilon=epsilon,
    early_stop=early_stop,
    attackable_features="time_features",
    enable_time_shifting=True
)

print(f"\nAttack completed!")
print(f"Total attacks performed: {len(results)}")
print(f"Successful attacks: {sum(1 for r in results.values() if r['success'])}")
print(f"Failed attacks: {sum(1 for r in results.values() if not r['success'])}")


Starting gradient ascent attacks...
Total prefix-suffix pairs to attack: 1898


Performing gradient ascent attacks:   1%|          | 13/1898 [00:00<00:14, 126.86it/s]

TIME SHIFTING ENABLED


Performing gradient ascent attacks:   5%|▌         | 101/1898 [00:00<00:08, 201.88it/s]

TIME SHIFTING ENABLED
TIME SHIFTING ENABLED


...

TIME SHIFTING ENABLED


Performing gradient ascent attacks:  97%|█████████▋| 1836/1898 [00:10<00:00, 207.19it/s]

TIME SHIFTING ENABLED


Performing gradient ascent attacks: 100%|██████████| 1898/1898 [00:10<00:00, 177.02it/s]

TIME SHIFTING ENABLED

Attack completed!
Total attacks performed: 71
Successful attacks: 2
Failed attacks: 69


In [8]:
# Save results
if len(results) > 0:
    # Save all results at once, or in chunks if needed
    if len(results) <= save_every:
        # Save all at once
        filename = os.path.join(output_dir, 'gradient_ascent_attack_all.pkl')
        with open(filename, 'wb') as f:
            pickle.dump(results, f)
        print(f"Saved all {len(results)} results to {filename}")
    else:
        # Save in chunks
        results_list = list(results.items())
        for i in range(0, len(results_list), save_every):
            chunk = dict(results_list[i:i+save_every])
            chunk_number = (i // save_every) + 1
            save_chunk(chunk, chunk_number)
        print(f"Saved {len(results)} results in chunks")
else:
    print("No results to save")


Saved 50 results to ../../../../../evaluation_results/robustness/Helpdesk/gradient_ascent_attack/gradient_ascent_attack_part_0001.pkl
Saved 21 results to ../../../../../evaluation_results/robustness/Helpdesk/gradient_ascent_attack/gradient_ascent_attack_part_0002.pkl
Saved 71 results in chunks


In [9]:
# Print summary statistics
if len(results) > 0:
    successful_attacks = [r for r in results.values() if r['success']]
    failed_attacks = [r for r in results.values() if not r['success']]
    
    print("\n=== Attack Summary ===")
    print(f"Total attacks: {len(results)}")
    print(f"Successful attacks: {len(successful_attacks)}")
    print(f"Failed attacks: {len(failed_attacks)}")
    
    if successful_attacks:
        num_steps = [r['num_steps'] for r in successful_attacks]
        print(f"\nSuccessful attack statistics:")
        print(f"  Average steps: {sum(num_steps) / len(num_steps):.2f}")
        print(f"  Min steps: {min(num_steps)}")
        print(f"  Max steps: {max(num_steps)}")
    
    if failed_attacks:
        num_steps_failed = [r['num_steps'] for r in failed_attacks]
        print(f"\nFailed attack statistics:")
        print(f"  Average steps: {sum(num_steps_failed) / len(num_steps_failed):.2f}")
        print(f"  All reached max iterations: {all(n == max_iterations for n in num_steps_failed)}")
else:
    print("No results to summarize")



=== Attack Summary ===
Total attacks: 71
Successful attacks: 2
Failed attacks: 69

Successful attack statistics:
  Average steps: 1.00
  Min steps: 1
  Max steps: 1

Failed attack statistics:
  Average steps: 1.00
  All reached max iterations: True


In [10]:
# Example: Inspect a few attack results
if len(results) > 0:
    print("\n=== Example Attack Results ===")
    
    # Show first few successful attacks
    successful = [(k, v) for k, v in results.items() if v['success']]
    if successful:
        print(f"\nFirst successful attack:")
        (case_id, prefix_len), result = successful[0]
        print(f"  Case ID: {case_id}, Prefix Length: {prefix_len}")
        print(f"  Steps taken: {result['num_steps']}")
        print(f"  Original suffix length: {len(result['original_suffix'])}")
        print(f"  Perturbed suffix length: {len(result['perturbed_suffix'])}")
        
        # Show activity sequences
        if result['original_suffix'] and result['perturbed_suffix']:
            orig_activities = [e.get('Activity', 'N/A') for e in result['original_suffix']]
            pert_activities = [e.get('Activity', 'N/A') for e in result['perturbed_suffix']]
            print(f"  Original activities: {orig_activities}")
            print(f"  Perturbed activities: {pert_activities}")
    
    # Show first few failed attacks
    failed = [(k, v) for k, v in results.items() if not v['success']]
    if failed:
        print(f"\nFirst failed attack:")
        (case_id, prefix_len), result = failed[0]
        print(f"  Case ID: {case_id}, Prefix Length: {prefix_len}")
        print(f"  Steps taken: {result['num_steps']}")
        print(f"  Note: Attack did not succeed within {max_iterations} iterations")
else:
    print("No results to inspect")



=== Example Attack Results ===

First successful attack:
  Case ID: Case 1254, Prefix Length: 4
  Steps taken: 1
  Original suffix length: 2
  Perturbed suffix length: 2
  Original activities: ['Resolve ticket', 'Closed']
  Perturbed activities: ['Closed', 'Closed']

First failed attack:
  Case ID: Case 10, Prefix Length: 2
  Steps taken: 1
  Note: Attack did not succeed within 1 iterations


In [11]:
# Print before/after prefix and suffix for all attack candidates
if len(results) > 0:
    print("\n" + "="*80)
    print("BEFORE/AFTER PREFIX AND SUFFIX FOR ALL ATTACK CANDIDATES")
    print("="*80)
    
    for idx, ((case_id, prefix_len), result) in enumerate(results.items(), 1):
        print(f"\n{'='*80}")
        print(f"Attack #{idx}: Case ID: {case_id}, Prefix Length: {prefix_len}")
        print(f"Status: {'SUCCESS' if result['success'] else 'FAILED'}")
        print(f"Steps taken: {result['num_steps']}")
        print(f"{'='*80}")
        
        # Convert original prefix from tensor to readable format
        original_prefix_readable = attacker.case_to_readable(
            (result['original_prefix'][0], result['original_prefix'][1]), 
            prune_eos=True
        )
        
        # Convert perturbed prefix from tensor to readable format
        perturbed_prefix_readable = attacker.case_to_readable(
            (result['perturbed_prefix'][0], result['perturbed_prefix'][1]), 
            prune_eos=True
        )
        
        # Print PREFIX with clean and perturbed values side-by-side
        print(f"\n--- PREFIX COMPARISON (Length: {len(original_prefix_readable)}) ---")
        max_prefix_len = max(len(original_prefix_readable), len(perturbed_prefix_readable))
        for i in range(max_prefix_len):
            print(f"\n  Event {i+1}:")
            orig_event = original_prefix_readable[i] if i < len(original_prefix_readable) else {}
            pert_event = perturbed_prefix_readable[i] if i < len(perturbed_prefix_readable) else {}
            
            # Get all unique keys from both events
            all_keys = set(orig_event.keys()) | set(pert_event.keys())
            
            for key in sorted(all_keys):
                orig_value = orig_event.get(key, 'N/A')
                pert_value = pert_event.get(key, 'N/A')
                
                # Highlight if values differ
                if orig_value != pert_value:
                    print(f"    {key} = [{orig_value}] -> [{pert_value}] ⚠️ CHANGED")
                else:
                    print(f"    {key} = [{orig_value}], [{pert_value}]")
        
        # Print SUFFIX with clean and perturbed values side-by-side
        print(f"\n--- SUFFIX COMPARISON ---")
        orig_suffix = result['original_suffix']
        pert_suffix = result['perturbed_suffix']
        max_suffix_len = max(len(orig_suffix), len(pert_suffix))
        
        for i in range(max_suffix_len):
            orig_event = orig_suffix[i] if i < len(orig_suffix) else {}
            pert_event = pert_suffix[i] if i < len(pert_suffix) else {}
            
            # Get all unique keys from both events
            all_keys = set(orig_event.keys()) | set(pert_event.keys())
            
            print(f"\n  Event {i+1}:")
            for key in sorted(all_keys):
                orig_value = orig_event.get(key, 'N/A')
                pert_value = pert_event.get(key, 'N/A')
                
                # Highlight if values differ
                if orig_value != pert_value:
                    print(f"    {key} = [{orig_value}] -> [{pert_value}] ⚠️ CHANGED")
                else:
                    print(f"    {key} = [{orig_value}], [{pert_value}]")
        
        # Activity sequence summary
        orig_prefix_activities = [e.get('Activity', 'N/A') for e in original_prefix_readable]
        pert_prefix_activities = [e.get('Activity', 'N/A') for e in perturbed_prefix_readable]
        orig_suffix_activities = [e.get('Activity', 'N/A') for e in result['original_suffix']]
        pert_suffix_activities = [e.get('Activity', 'N/A') for e in result['perturbed_suffix']]
        
        print(f"\n--- ACTIVITY SEQUENCE SUMMARY ---")
        print(f"Prefix activities: {orig_prefix_activities} -> {pert_prefix_activities}")
        print(f"Suffix activities: {orig_suffix_activities} -> {pert_suffix_activities}")
        
        # Check if prefix changed
        prefix_changed = orig_prefix_activities != pert_prefix_activities
        print(f"\nPrefix changed: {prefix_changed}")
        if prefix_changed:
            print("  Positions changed:")
            for i, (orig, pert) in enumerate(zip(orig_prefix_activities, pert_prefix_activities)):
                if orig != pert:
                    print(f"    Position {i+1}: '{orig}' -> '{pert}'")
        
        # Check if suffix changed
        suffix_changed = orig_suffix_activities != pert_suffix_activities
        print(f"Suffix changed: {suffix_changed}")
        if suffix_changed:
            print("  Positions changed:")
            min_len = min(len(orig_suffix_activities), len(pert_suffix_activities))
            for i in range(min_len):
                if orig_suffix_activities[i] != pert_suffix_activities[i]:
                    print(f"    Position {i+1}: '{orig_suffix_activities[i]}' -> '{pert_suffix_activities[i]}'")
            if len(orig_suffix_activities) != len(pert_suffix_activities):
                print(f"  Length difference: {len(orig_suffix_activities)} vs {len(pert_suffix_activities)}")
        
        print(f"\n{'-'*80}")
    
    print(f"\n{'='*80}")
    print(f"Total attacks printed: {len(results)}")
    print(f"{'='*80}")
else:
    print("No results to print")



BEFORE/AFTER PREFIX AND SUFFIX FOR ALL ATTACK CANDIDATES

Attack #1: Case ID: Case 10, Prefix Length: 2
Status: FAILED
Steps taken: 1

--- PREFIX COMPARISON (Length: 2) ---

  Event 1:
    Activity = [Assign seriousness], [Assign seriousness]
    Resource = [Value 2], [Value 2]
    Variant index = [1.0], [1.0]
    case_elapsed_time = [0.01788514363579452], [0.01788514363579452]
    customer = [Value 10], [Value 10]
    day_in_week = [1.9999999773389212], [1.9999999773389212]
    event_elapsed_time = [974937.1249999986] -> [936499.2862269109] ⚠️ CHANGED
    product = [Value 3], [Value 3]
    responsible_section = [Value 2], [Value 2]
    seconds_in_day = [31819.99974958774], [31819.99974958774]
    seriousness = [Value 1], [Value 1]
    seriousness_2 = [Value 1], [Value 1]
    service_level = [Value 3], [Value 3]
    service_type = [Value 1], [Value 1]
    support_section = [Value 2], [Value 2]
    workgroup = [Value 3], [Value 3]

  Event 2:
    Activity = [Take in charge ticket], [Ta